# **XGBoost**



In [2]:
import pandas as pd # Para cargar los datos y hacer OHE.
import numpy as np  # Para lidiar con NaNs.
import time
import base
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import confusion_matrix



random_state = 42
np.random.seed(random_state)

In [3]:
train_data = pd.read_csv("data/train_data.txt", sep="\t", low_memory=False, on_bad_lines='skip')
test_data = pd.read_csv("data/test_data.txt", sep="\t", low_memory=False, on_bad_lines='skip')
print(train_data.shape)
print(test_data.shape)

(911344, 22)
(51570, 21)


In [4]:
train_data.head()


,ts,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_end,shuffle,offline,offline_timestamp,incognito_mode,username,obs_id
0,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,The Eater Of Dreams,Nine Inch Nails,Hesitation Marks,spotify:track:1IPdwxRUbuNZiRpFN49RQC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwdbtn,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,1
1,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Copy Of A,Nine Inch Nails,Hesitation Marks,spotify:track:4BFKCEp4gwG3QHNlYodLMy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,2
2,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,El extranjero,Bunbury,Singles,spotify:track:7ghPo5JVqcmwZQSpm6uyGG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,trackdone,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,3
3,2013-10-29 18:20:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Luna,Bunbury,Singles,spotify:track:4g0Jk6sWxdu3FwwGDtoQPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwdbtn,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,4
4,2013-10-29 18:40:00+00:00,Windows 7 (6.1.7601; x86; SP1; S),AR,6472d74d7192fecaa2744625ea9e29285bde602e641a03...,Feliz año,Bunbury,Singles,spotify:track:0GZzkyz7MN15llCyCBgPWh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwdbtn,False,False,NaN,False,4324517c6925bba98b4e3a6896d1398fae8f777969e7bc...,5


In [6]:
train_data.describe()

,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,obs_id
count,0.0,0.0,0.0,0.0,911344.0000
mean,NaN,NaN,NaN,NaN,455672.5000
std,NaN,NaN,NaN,NaN,263082.4962
min,NaN,NaN,NaN,NaN,1.0000
25%,NaN,NaN,NaN,NaN,227836.7500
50%,NaN,NaN,NaN,NaN,455672.5000
75%,NaN,NaN,NaN,NaN,683508.2500
max,NaN,NaN,NaN,NaN,911344.0000


In [7]:
train_data.describe(include = 'object')

,ts,platform,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_end,offline_timestamp,username
count,911344,911344,911344,911344,907211,907211,907211,907211,4133,4133,4133,911344,200888,911344
unique,113343,570,43,18444,95837,18543,41445,129731,1846,349,1861,15,24346,10
top,2019-02-07 21:40:00+00:00,ios,AR,d28b0c9777b108ac7f5ab057c4a7da93a8e2c2bf0c941a...,I Feel It Coming,Patricio Rey y sus Redonditos de Ricota,Nancy & Lee,spotify:track:1jJci4qxiYcOHhQR247rEU,La Zorra y el Conejo,Las Nuevas Historias de CoComelon,spotify:episode:42geAoXFSiFulSIRpfy03e,trackdone,1970-01-01 00:00:00+00:00,f15bafb0dc3331899681fffc6fbf48c10576b10e4a9044...
freq,424,79881,816259,16465,746,10912,3386,585,41,589,41,521220,2036,222644


In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911344 entries, 0 to 911343
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 911344 non-null  object 
 1   platform                           911344 non-null  object 
 2   conn_country                       911344 non-null  object 
 3   ip_addr                            911344 non-null  object 
 4   master_metadata_track_name         907211 non-null  object 
 5   master_metadata_album_artist_name  907211 non-null  object 
 6   master_metadata_album_album_name   907211 non-null  object 
 7   spotify_track_uri                  907211 non-null  object 
 8   episode_name                       4133 non-null    object 
 9   episode_show_name                  4133 non-null    object 
 10  spotify_episode_uri                4133 non-null    object 
 11  audiobook_title                    0 no

In [ ]:
# Importante: ¡sólo poner las categóricas y excluir la variable a predecir!
pd_ohe = pd.get_dummies(train_data,
                        columns = ['ts', 'platform', 'conn_country', 'ip_addr', 'master_metadata_track_name',
                                   'master_metadata_album_artist_name', 'master_metadata_album_album_name',
                                    'spotify_track_uri', 'username'],
                        sparse = True,    # Devolver una matriz rala.
                        dummy_na = False, # No agregar columna para NaNs.
                        dtype = int       # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )
pd_ohe

In [ ]:
len(pd_ohe.columns)

NameError: name 'pd_ohe' is not defined

In [ ]:
pd_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 35 columns):
 #   Column                                                                     Non-Null Count  Dtype           
---  ------                                                                     --------------  -----           
 0   episode_name                                                               0 non-null      float64         
 1   episode_show_name                                                          0 non-null      float64         
 2   spotify_episode_uri                                                        0 non-null      float64         
 3   audiobook_title                                                            0 non-null      float64         
 4   audiobook_uri                                                              0 non-null      float64         
 5   audiobook_chapter_uri                                                      0 non-null      float64     

C:\Users\gabit\AppData\Local\Temp\ipykernel_13716\3289545234.py:1: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  pd_ohe.info()


### **Preparar conjuntos de entrenamiento, validación (*hold-out*) y evaluación**

In [ ]:
# fwdbtn => 1 (skip), otro => 0
pd_ohe['y'] = (train_data['reason_end'] == 'fwdbtn').astype(int)

y = pd_ohe[['y']].copy()  # mantenemos tu patrón
y


,y
0,1
1,0
2,0
3,1
4,1


In [ ]:
#saco el mapeo yes/no por 0/1
y['y'] = y['y'].astype(int)


In [ ]:
y['y'].unique()

array([1, 0])

In [ ]:
X = pd_ohe.drop('y', axis = 1)
X

,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,shuffle,offline,offline_timestamp,incognito_mode,obs_id,ts_2013-10-29 18:20:00+00:00,ts_2013-10-29 18:40:00+00:00,platform_Windows 7 (6.1.7601; x86; SP1; S),conn_country_AR,ip_addr_6472d74d7192fecaa2744625ea9e29285bde602e641a033f2bca0c4ac33835e8,master_metadata_track_name_Copy Of A,master_metadata_track_name_El extranjero,master_metadata_track_name_Feliz año,master_metadata_track_name_Luna,master_metadata_track_name_The Eater Of Dreams,master_metadata_album_artist_name_Bunbury,master_metadata_album_artist_name_Nine Inch Nails,master_metadata_album_album_name_Hesitation Marks,master_metadata_album_album_name_Singles,spotify_track_uri_spotify:track:0GZzkyz7MN15llCyCBgPWh,spotify_track_uri_spotify:track:1IPdwxRUbuNZiRpFN49RQC,spotify_track_uri_spotify:track:4BFKCEp4gwG3QHNlYodLMy,spotify_track_uri_spotify:track:4g0Jk6sWxdu3FwwGDtoQPL,spotify_track_uri_spotify:track:7ghPo5JVqcmwZQSpm6uyGG,reason_end_fwdbtn,reason_end_trackdone,reason_end_unknown,username_4324517c6925bba98b4e3a6896d1398fae8f777969e7bce54aa460ba9a40358b
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,False,1,1,0,1,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,False,2,1,0,1,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,False,3,1,0,1,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,False,4,1,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,False,5,0,1,1,1,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1


In [ ]:
#cambio y por y['y']
val_test_size = 0.3  # Proporción de la suma del test de validación y del de test.

X_train, X_tmp, Y_train, Y_tmp = train_test_split(X, y['y'],
                                train_size=0.7,
                                random_state=random_state,
                                stratify=y['y']
)



In [ ]:
X_val, X_test, Y_val, Y_test = train_test_split(X_tmp, Y_tmp,
                                                train_size=0.5,
                                                random_state=random_state,
                                                stratify=Y_tmp
)


In [ ]:
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')
print(f'Cantidad de datos de test: {len(X_test)}')

## XGBoost

In [ ]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic',
                            seed = random_state,
                            eval_metric = 'auc')



In [ ]:
clf_xgb.fit(X_train, Y_train, verbose = True, eval_set = [(X_val, Y_val)])

### **Búsqueda de hiperparámetros**

In [ ]:
from scipy.stats import uniform
params = {'max_depth': list(range(1, 40)),
          'learning_rate': uniform(scale = 0.2),
          'gamma': uniform(scale = 2),
          'reg_lambda': uniform(scale = 5),        # Parámetro de regularización.
          'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
          'min_child_weight': uniform(scale = 5),
          'colsample_bytree': uniform(0.75, 0.25), # Entre 0.75 y 1.
          'n_estimators': list(range(1, 1000))
         }

In [ ]:
start = time.time()
best_score = 0
best_estimator = None
iterations = 100
for g in ParameterSampler(params, n_iter = iterations, random_state = random_state):
    clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **g)
    clf_xgb.fit(X_train, Y_train, eval_set = [(X_val, Y_val)], verbose = False)

    y_pred = clf_xgb.predict_proba(X_val)[:, 1] # Obtenemos la probabilidad de una de las clases (cualquiera).
    auc_roc = sklearn.metrics.roc_auc_score(Y_val, y_pred)
    # Guardamos si es mejor.
    if auc_roc > best_score:
        print(f'Mejor valor de ROC-AUC encontrado: {auc_roc}')
        best_score = auc_roc
        best_grid = g
        best_estimator = clf_xgb

end = time.time()
print('ROC-AUC: %0.5f' % best_score)
print('Grilla:', best_grid)
print(f'Tiempo transcurrido: {str(end - start)} segundos')
print(f'Tiempo de entrenamiento por iteración: {str(round((end - start) / iterations, 2))} segundos')

In [ ]:
best_grid = {'colsample_bytree': 0.7626921327598493,
             'gamma': 1.7732342979013198,
             'learning_rate': 0.005523354374740941,
             'max_depth': 34,
             'min_child_weight': 0.469909699204345,
             'n_estimators': 727,
             'reg_lambda': 3.360130676475997,
             'subsample': 0.664076333737366
            }
# Aclaración: acá está "hardcodeado", pero se puede hacer mejor, accediendo a los valores de `best_grid`.

best_estimator = xgb.XGBClassifier(objective = 'binary:logistic',
                                   seed = random_state,
                                   eval_metric = 'auc',
                                   **best_grid)

best_estimator.fit(X_train, Y_train, verbose = True,  eval_set = [(X_val, Y_val)])

# roc_auc_score requiere un array 1D; da lo mismo qué dimensión le pasemos: 90 o 1.
y_pred = best_estimator.predict_proba(X_val)[:, 1]
auc_roc = sklearn.metrics.roc_auc_score(Y_val, y_pred)
print('AUC-ROC validación: %0.5f' % auc_roc)

In [ ]:
bst = best_estimator.get_booster()
for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
    print('%s: ' % importance_type, bst.get_score(importance_type = importance_type))
    print('--------------')

### **Conjunto de test**

In [ ]:
y_pred = best_estimator.predict_proba(X_test)[:, 1]
auc_roc = sklearn.metrics.roc_auc_score(Y_test, y_pred)
print('AUC-ROC test: %0.5f' % auc_roc)